In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import ast
from datetime import datetime, timedelta, timezone
from collections import Counter
from itertools import chain
from collections import defaultdict

In [57]:
# reading csv file and storing it in a dataframe
listings_df = pd.read_csv("listings_data.csv")
all_data_df = pd.read_csv('all_data.csv')
hist_df = pd.read_csv('historical_data.csv')

In [58]:
listings_df.head(10)

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
0,1,Bitcoin,BTC,bitcoin,12187,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",2.100000e+07,1.986980e+07,1.986980e+07,...,-1.347124,2.810681,12.757457,24.610176,13.927691,2.135314e+12,63.2857,2.256771e+12,NaN,2025-05-25T19:03:00.000Z
1,1027,Ethereum,ETH,ethereum,10307,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,1.207260e+08,1.207260e+08,...,-1.414252,2.107912,39.794561,26.117982,-5.148470,3.038475e+11,9.0051,3.038475e+11,NaN,2025-05-25T19:03:00.000Z
2,825,Tether USDt,USDT,tether,129577,2015-02-25T00:00:00.000Z,"['stablecoin', 'asset-backed-stablecoin', 'usd...",NaN,1.527330e+11,1.560047e+11,...,0.016888,0.004997,-0.014991,0.028329,-0.013933,1.527815e+11,4.5280,1.560543e+11,NaN,2025-05-25T19:02:00.000Z
3,52,XRP,XRP,xrp,1614,2013-08-04T00:00:00.000Z,"['medium-of-exchange', 'enterprise-solutions',...",1.000000e+11,5.868601e+10,9.998611e+10,...,-1.849945,-4.035482,4.963800,-2.963648,-5.979773,1.351875e+11,4.0065,2.303573e+11,NaN,2025-05-25T19:03:00.000Z
4,1839,BNB,BNB,bnb,2545,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,1.408881e+08,1.408881e+08,...,-1.484729,3.304452,10.219053,7.628989,4.982428,9.374372e+10,2.7783,9.374372e+10,NaN,2025-05-25T19:02:00.000Z
5,5426,Solana,SOL,solana,924,2020-04-10T00:00:00.000Z,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",NaN,5.202837e+08,6.015333e+08,...,-2.954381,0.757709,13.302747,24.985409,12.985334,8.934963e+10,2.6480,1.033028e+11,NaN,2025-05-25T19:03:00.000Z
6,3408,USDC,USDC,usd-coin,28186,2018-10-08T00:00:00.000Z,"['medium-of-exchange', 'stablecoin', 'asset-ba...",NaN,6.171563e+10,6.171563e+10,...,0.001908,-0.006457,-0.003849,-0.002124,-0.006248,6.170706e+10,1.8289,6.170706e+10,NaN,2025-05-25T19:03:00.000Z
7,74,Dogecoin,DOGE,dogecoin,1227,2013-12-15T00:00:00.000Z,"['mineable', 'pow', 'scrypt', 'medium-of-excha...",NaN,1.494131e+11,1.494131e+11,...,-3.302346,-3.798250,20.798547,13.025218,-2.734797,3.278373e+10,0.9716,3.278373e+10,NaN,2025-05-25T19:03:00.000Z
8,2010,Cardano,ADA,cardano,1521,2017-10-01T00:00:00.000Z,"['dpos', 'pos', 'platform', 'research', 'smart...",4.500000e+10,3.533451e+10,4.499450e+10,...,-1.176673,0.065855,3.763610,2.899820,2.716070,2.638633e+10,0.7820,3.360410e+10,NaN,2025-05-25T19:03:00.000Z
9,1958,TRON,TRX,tron,1178,2017-09-13T00:00:00.000Z,"['media', 'payments', 'tron-ecosystem', 'layer...",NaN,9.487783e+10,9.487783e+10,...,0.003916,2.114464,11.740144,17.936722,9.747367,2.572893e+10,0.7625,2.572893e+10,NaN,2025-05-25T19:03:00.000Z


In [62]:
# checking data type of each column
listings_df.dtypes

id                                    int64
name                                 object
symbol                               object
slug                                 object
num_market_pairs                      int64
date_added                           object
tags                                 object
max_supply                          float64
circulating_supply                  float64
total_supply                        float64
infinite_supply                        bool
platform                             object
cmc_rank                              int64
self_reported_circulating_supply    float64
self_reported_market_cap            float64
tvl_ratio                           float64
last_updated                         object
price                               float64
volume_24h                          float64
volume_change_24h                   float64
percent_change_1h                   float64
percent_change_24h                  float64
percent_change_7d               

In [4]:
# tags column contain stringified lists and not actual lists
print(listings_df['tags'].apply(type).value_counts())

tags
<class 'str'>    5000
Name: count, dtype: int64


In [5]:
# converting tags column to actual list
listings_df['tags'] = listings_df['tags'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) and x.startswith('[') else []
)

all_data_df['tags'] = all_data_df['tags'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) and x.startswith('[') else []
)

hist_df['tags'] = hist_df['tags'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) and x.startswith('[') else []
)


In [6]:
print(hist_df['tags'].apply(type).value_counts())

tags
<class 'list'>    540
Name: count, dtype: int64


In [7]:
# adding date columns
date_cols = ['date_added', 'last_updated', 'last_updated.1' ]

# changing date columns to datetime object
for column in date_cols:
    if column in listings_df.columns:
        listings_df[column] = pd.to_datetime(listings_df[column])
    else:
        print(f"Column '{column}' not found in DataFrame.")
        
        
listings_df.dtypes

id                                                int64
name                                             object
symbol                                           object
slug                                             object
num_market_pairs                                  int64
date_added                          datetime64[ns, UTC]
tags                                             object
max_supply                                      float64
circulating_supply                              float64
total_supply                                    float64
infinite_supply                                    bool
platform                                         object
cmc_rank                                          int64
self_reported_circulating_supply                float64
self_reported_market_cap                        float64
tvl_ratio                                       float64
last_updated                        datetime64[ns, UTC]
price                                           

In [8]:
# drop duplicates
listings_df.drop_duplicates(subset=["id"])

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
0,1,Bitcoin,BTC,bitcoin,12187,2010-07-13 00:00:00+00:00,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.986980e+07,1.986980e+07,...,-1.347124,2.810681,12.757457,24.610176,13.927691,2.135314e+12,63.2857,2.256771e+12,NaN,2025-05-25 19:03:00+00:00
1,1027,Ethereum,ETH,ethereum,10307,2015-08-07 00:00:00+00:00,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207260e+08,1.207260e+08,...,-1.414252,2.107912,39.794561,26.117982,-5.148470,3.038475e+11,9.0051,3.038475e+11,NaN,2025-05-25 19:03:00+00:00
2,825,Tether USDt,USDT,tether,129577,2015-02-25 00:00:00+00:00,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,1.527330e+11,1.560047e+11,...,0.016888,0.004997,-0.014991,0.028329,-0.013933,1.527815e+11,4.5280,1.560543e+11,NaN,2025-05-25 19:02:00+00:00
3,52,XRP,XRP,xrp,1614,2013-08-04 00:00:00+00:00,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.868601e+10,9.998611e+10,...,-1.849945,-4.035482,4.963800,-2.963648,-5.979773,1.351875e+11,4.0065,2.303573e+11,NaN,2025-05-25 19:03:00+00:00
4,1839,BNB,BNB,bnb,2545,2017-07-25 00:00:00+00:00,"[marketplace, centralized-exchange, payments, ...",NaN,1.408881e+08,1.408881e+08,...,-1.484729,3.304452,10.219053,7.628989,4.982428,9.374372e+10,2.7783,9.374372e+10,NaN,2025-05-25 19:02:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,25712,AiAkita,AIAKITA,aiakita,3,2023-05-22 21:50:37+00:00,[arbitrum-ecosystem],3.139079e+17,0.000000e+00,3.139079e+17,...,1.668990,-2.217800,-27.547658,-25.165560,-38.338088,0.000000e+00,0.0000,3.265387e+04,NaN,2025-05-25 19:02:00+00:00
4996,36565,$SEND,$SEND,send,1,2025-05-20 09:58:34+00:00,"[memes, solana-ecosystem]",1.000000e+17,0.000000e+00,1.000000e+17,...,-36.750024,-96.188138,-96.188138,-96.188138,-96.188138,0.000000e+00,0.0000,2.566771e+04,NaN,2025-05-25 19:03:00+00:00
4997,35067,BCGame Coin,BC,bcgame-coin,2,2025-01-07 06:53:23+00:00,[solana-ecosystem],NaN,0.000000e+00,1.000000e+10,...,-5.460912,102.958377,52.589969,74.535495,44.872525,0.000000e+00,0.0000,6.516923e+07,NaN,2025-05-25 19:03:00+00:00
4998,24439,Silver,KAG,silver,20,2023-04-18 14:22:39+00:00,[],NaN,0.000000e+00,1.000000e+11,...,-0.673155,3.011141,-1.620380,-1.467504,2.186950,0.000000e+00,0.0000,3.320438e+12,NaN,2025-05-25 19:03:00+00:00


In [9]:
# check for null values
listings_df.isnull().any()

id                                  False
name                                False
symbol                              False
slug                                False
num_market_pairs                    False
date_added                          False
tags                                False
max_supply                           True
circulating_supply                  False
total_supply                        False
infinite_supply                     False
platform                             True
cmc_rank                            False
self_reported_circulating_supply     True
self_reported_market_cap             True
tvl_ratio                            True
last_updated                        False
price                               False
volume_24h                          False
volume_change_24h                   False
percent_change_1h                   False
percent_change_24h                  False
percent_change_7d                   False
percent_change_30d                

In [10]:
# drop columns
listings_df = listings_df.drop(['platform'], axis=1)
listings_df.head(10)


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
0,1,Bitcoin,BTC,bitcoin,12187,2010-07-13 00:00:00+00:00,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.986980e+07,1.986980e+07,...,-1.347124,2.810681,12.757457,24.610176,13.927691,2.135314e+12,63.2857,2.256771e+12,NaN,2025-05-25 19:03:00+00:00
1,1027,Ethereum,ETH,ethereum,10307,2015-08-07 00:00:00+00:00,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207260e+08,1.207260e+08,...,-1.414252,2.107912,39.794561,26.117982,-5.148470,3.038475e+11,9.0051,3.038475e+11,NaN,2025-05-25 19:03:00+00:00
2,825,Tether USDt,USDT,tether,129577,2015-02-25 00:00:00+00:00,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,1.527330e+11,1.560047e+11,...,0.016888,0.004997,-0.014991,0.028329,-0.013933,1.527815e+11,4.5280,1.560543e+11,NaN,2025-05-25 19:02:00+00:00
3,52,XRP,XRP,xrp,1614,2013-08-04 00:00:00+00:00,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.868601e+10,9.998611e+10,...,-1.849945,-4.035482,4.963800,-2.963648,-5.979773,1.351875e+11,4.0065,2.303573e+11,NaN,2025-05-25 19:03:00+00:00
4,1839,BNB,BNB,bnb,2545,2017-07-25 00:00:00+00:00,"[marketplace, centralized-exchange, payments, ...",NaN,1.408881e+08,1.408881e+08,...,-1.484729,3.304452,10.219053,7.628989,4.982428,9.374372e+10,2.7783,9.374372e+10,NaN,2025-05-25 19:02:00+00:00
5,5426,Solana,SOL,solana,924,2020-04-10 00:00:00+00:00,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,5.202837e+08,6.015333e+08,...,-2.954381,0.757709,13.302747,24.985409,12.985334,8.934963e+10,2.6480,1.033028e+11,NaN,2025-05-25 19:03:00+00:00
6,3408,USDC,USDC,usd-coin,28186,2018-10-08 00:00:00+00:00,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,6.171563e+10,6.171563e+10,...,0.001908,-0.006457,-0.003849,-0.002124,-0.006248,6.170706e+10,1.8289,6.170706e+10,NaN,2025-05-25 19:03:00+00:00
7,74,Dogecoin,DOGE,dogecoin,1227,2013-12-15 00:00:00+00:00,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,1.494131e+11,1.494131e+11,...,-3.302346,-3.798250,20.798547,13.025218,-2.734797,3.278373e+10,0.9716,3.278373e+10,NaN,2025-05-25 19:03:00+00:00
8,2010,Cardano,ADA,cardano,1521,2017-10-01 00:00:00+00:00,"[dpos, pos, platform, research, smart-contract...",4.500000e+10,3.533451e+10,4.499450e+10,...,-1.176673,0.065855,3.763610,2.899820,2.716070,2.638633e+10,0.7820,3.360410e+10,NaN,2025-05-25 19:03:00+00:00
9,1958,TRON,TRX,tron,1178,2017-09-13 00:00:00+00:00,"[media, payments, tron-ecosystem, layer-1, dwf...",NaN,9.487783e+10,9.487783e+10,...,0.003916,2.114464,11.740144,17.936722,9.747367,2.572893e+10,0.7625,2.572893e+10,NaN,2025-05-25 19:03:00+00:00


In [11]:
# sort data according to price
price_df = listings_df.sort_values(by='price', ascending=False)
price_df.head(10)

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
3265,4077,Maya Preferred PRA,MPRA,maya-preferred,3,2019-07-03 00:00:00+00:00,[ethereum-ecosystem],200000000.0,0.000000e+00,2.000000e+08,...,0.011970,9.718939,102.988946,1194.675790,1194.675790,0.000000e+00,0.0000,2.040841e+15,NaN,2025-05-25 19:03:00+00:00
0,1,Bitcoin,BTC,bitcoin,12187,2010-07-13 00:00:00+00:00,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,1.986980e+07,1.986980e+07,...,-1.347124,2.810681,12.757457,24.610176,13.927691,2.135314e+12,63.2857,2.256771e+12,NaN,2025-05-25 19:03:00+00:00
4814,9486,Bitcoin TRC20,BTCT,bitcoin-trc20,7,2021-04-29 00:00:00+00:00,[tron20-ecosystem],21000000.0,0.000000e+00,2.500000e+04,...,-1.681241,2.664830,12.757108,23.786072,13.414569,0.000000e+00,0.0000,2.249931e+12,NaN,2025-05-25 19:03:00+00:00
3241,31042,Bitcoin Bridged ZED20,BTC.z,token-bitcoin-bridged-zed20,6,2024-05-02 13:01:00+00:00,[bnb-chain-ecosystem],500000.0,0.000000e+00,5.000000e+05,...,-1.246565,2.663084,12.696436,24.163669,13.824479,0.000000e+00,0.0000,5.333301e+10,NaN,2025-05-25 19:02:00+00:00
4702,36175,Universal BTC,UNIBTC,universal-btc,24,2025-04-01 04:54:23+00:00,"[ethereum-ecosystem, arbitrum-ecosystem, bnb-c...",NaN,0.000000e+00,1.147661e+02,...,-1.297174,1.672897,13.151689,28.248488,28.248488,0.000000e+00,0.0000,1.212713e+07,NaN,2025-05-25 19:03:00+00:00
3314,3759,Jinbi Token,JNB,jinbi-token,3,2019-02-22 00:00:00+00:00,[ethereum-ecosystem],NaN,0.000000e+00,1.250000e+07,...,0.008140,0.024779,42.805032,42.876219,79.840337,0.000000e+00,0.0000,6.945436e+11,NaN,2025-05-25 19:03:00+00:00
3430,21829,Ten Best Coins,TBC,ten-best-coins,2,2022-09-15 04:59:24+00:00,[ethereum-ecosystem],25000000.0,0.000000e+00,2.500000e+07,...,-1.493251,-0.237846,10.883048,9.243917,2.352600,0.000000e+00,0.0000,3.114031e+11,NaN,2025-05-25 19:03:00+00:00
208,28293,Ultima,ULTIMA,ultima,19,2023-10-26 20:59:30+00:00,[web3],100000.0,3.740900e+04,1.000000e+05,...,-18.982466,-21.467979,-41.234440,-46.358750,-39.293252,4.290220e+08,0.0000,1.146842e+09,NaN,2025-05-25 19:03:00+00:00
243,5864,yearn.finance,YFI,yearn-finance,534,2020-07-18 00:00:00+00:00,"[defi, dao, ethereum-ecosystem, yield-farming,...",NaN,3.381329e+04,3.664636e+04,...,-2.362624,-1.531347,4.000200,3.359969,-4.764194,1.858139e+08,0.0055,2.013824e+08,3.035298e+08,2025-05-25 19:02:00+00:00
4805,16388,Governance OHM,GOHM,governance-ohm,54,2021-12-23 02:41:20+00:00,"[ethereum-ecosystem, solana-ecosystem, terra-e...",NaN,0.000000e+00,1.131532e+05,...,-0.167473,-6.181052,-11.303366,-24.161135,-12.293357,0.000000e+00,0.0000,5.839868e+08,NaN,2025-05-25 19:03:00+00:00


In [12]:
# Locating name of the coin with index number 4256
listings_df.loc[4256,'name']

coin = listings_df.loc[listings_df['name'] == 'SmartMesh']
coin

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
4256,2277,SmartMesh,SMT,smartmesh,10,2017-12-13 00:00:00+00:00,"[iot, ethereum-ecosystem]",NaN,0.0,3.141593e+09,...,2.161937,10.119744,-9.490557,-54.471324,-61.458582,0.0,0.0,1143676.85,NaN,2025-05-25 19:03:00+00:00


In [13]:
# listing only coins with positive change in the last 90 days
positive_90_days_df = listings_df[listings_df['percent_change_90d']>0].sort_values(by='percent_change_90d', ascending=True)
positive_90_days_df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
53,26081,First Digital USD,FDUSD,first-digital-usd,524,2023-07-26 04:26:40+00:00,"[stablecoin, binance-smart-chain, usd-stableco...",NaN,1.646267e+09,1.646267e+09,...,-0.029289,-1.123206e-02,-5.632722e-02,4.070299e-02,1.707380e-03,1.643961e+09,0.0487,1.643961e+09,NaN,2025-05-25 19:03:00+00:00
655,7236,Celo Dollar,CUSD,celo-dollar,127,2020-09-29 00:00:00+00:00,"[defi, stablecoin, algorithmic-stablecoin, cel...",1.000000e+15,3.555417e+07,5.724157e+07,...,0.027749,-1.075850e-03,3.020500e-04,-9.033600e-04,2.788670e-03,3.555126e+07,0.0011,9.999183e+14,NaN,2025-05-25 19:03:00+00:00
27,29470,Ethena USDe,USDe,ethena-usde,153,2024-02-20 11:05:01+00:00,"[stablecoin, asset-backed-stablecoin, binance-...",NaN,4.996929e+09,4.996929e+09,...,-0.005504,2.478243e-02,9.296821e-02,1.031983e-01,8.025000e-03,5.000185e+09,0.1482,5.000185e+09,NaN,2025-05-25 19:03:00+00:00
1271,35372,StablR USD,USDR,stablr-usd,56,2025-02-17 05:40:36+00:00,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,5.850474e+06,5.850474e+06,...,-0.070002,9.995550e-03,-8.545663e-02,1.532094e-01,1.104337e-02,5.844714e+06,0.0000,5.844714e+06,NaN,2025-05-25 19:03:00+00:00
3620,9021,Wrapped XDAI,wxDai,wxdai,171,2021-03-30 00:00:00+00:00,[gnosis-chain-ecosystem],NaN,0.000000e+00,0.000000e+00,...,0.004371,2.730824e-02,1.217970e-03,-8.132640e-03,1.567257e-02,0.000000e+00,0.0000,0.000000e+00,NaN,2025-05-25 19:03:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3842,33647,Moo Deng (moodeng.sbs),MOODENG,moo-deng-sbs,1,2024-10-25 09:50:23+00:00,"[memes, ethereum-ecosystem, animal-memes]",1.000000e+12,0.000000e+00,1.000000e+12,...,509.847611,2.219387e+04,1.680584e+04,3.654898e+03,2.133492e+05,0.000000e+00,0.0000,3.259640e+07,NaN,2025-05-25 19:03:00+00:00
1375,35964,Mint Token,MT,mint-token,2,2025-03-07 07:45:06+00:00,"[platform, communications-social-media, dapp, ...",1.000000e+09,6.389244e+08,6.389244e+08,...,-1.568817,-8.796309e-01,3.250193e+00,2.272887e+01,2.524577e+05,4.520847e+06,0.0000,7.075716e+06,NaN,2025-05-25 19:03:00+00:00
4423,2502,Huobi Token,HT,htx-token,136,2018-02-03 00:00:00+00:00,"[marketplace, centralized-exchange, discount-t...",5.000000e+08,0.000000e+00,2.039804e+08,...,-7.630395,-1.157595e+01,-2.610899e+01,-3.230994e+01,4.771636e+06,0.000000e+00,0.0000,1.532264e+08,NaN,2025-05-25 19:03:00+00:00
3295,30953,ShibaBitcoin,SBBTC,shibabitcoin,9,2024-04-26 11:08:10+00:00,"[memes, bnb-chain-ecosystem]",1.000000e+09,0.000000e+00,1.000000e+09,...,0.210357,-9.281042e-01,1.522686e+01,6.027002e+06,6.016149e+06,0.000000e+00,0.0000,9.644272e+10,NaN,2025-05-25 19:03:00+00:00


In [14]:
# exclude coins with the memes and base-ecosystem tag 
exclude_list=['memes','base-ecosystem']
positive_90_days_df = positive_90_days_df[~positive_90_days_df['tags'].apply(lambda x: any(tag in exclude_list for tag in x))]

# exclude empty lists
positive_90_days_df = positive_90_days_df[listings_df['tags'].apply(len) > 0]


positive_90_days_df.reset_index(drop=True)

C:\Users\tohsh\AppData\Local\Temp\ipykernel_10752\372800653.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_90_days_df = positive_90_days_df[listings_df['tags'].apply(len) > 0]


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
0,26081,First Digital USD,FDUSD,first-digital-usd,524,2023-07-26 04:26:40+00:00,"[stablecoin, binance-smart-chain, usd-stableco...",NaN,1.646267e+09,1.646267e+09,...,-0.029289,-0.011232,-0.056327,0.040703,1.707380e-03,1.643961e+09,0.0487,1.643961e+09,NaN,2025-05-25 19:03:00+00:00
1,7236,Celo Dollar,CUSD,celo-dollar,127,2020-09-29 00:00:00+00:00,"[defi, stablecoin, algorithmic-stablecoin, cel...",1.000000e+15,3.555417e+07,5.724157e+07,...,0.027749,-0.001076,0.000302,-0.000903,2.788670e-03,3.555126e+07,0.0011,9.999183e+14,NaN,2025-05-25 19:03:00+00:00
2,29470,Ethena USDe,USDe,ethena-usde,153,2024-02-20 11:05:01+00:00,"[stablecoin, asset-backed-stablecoin, binance-...",NaN,4.996929e+09,4.996929e+09,...,-0.005504,0.024782,0.092968,0.103198,8.025000e-03,5.000185e+09,0.1482,5.000185e+09,NaN,2025-05-25 19:03:00+00:00
3,35372,StablR USD,USDR,stablr-usd,56,2025-02-17 05:40:36+00:00,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,5.850474e+06,5.850474e+06,...,-0.070002,0.009996,-0.085457,0.153209,1.104337e-02,5.844714e+06,0.0000,5.844714e+06,NaN,2025-05-25 19:03:00+00:00
4,9021,Wrapped XDAI,wxDai,wxdai,171,2021-03-30 00:00:00+00:00,[gnosis-chain-ecosystem],NaN,0.000000e+00,0.000000e+00,...,0.004371,0.027308,0.001218,-0.008133,1.567257e-02,0.000000e+00,0.0000,0.000000e+00,NaN,2025-05-25 19:03:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,33561,trumpwifhat,TRUMP,trump-wif-hat,1,2024-10-22 11:39:18+00:00,[ethereum-ecosystem],9.988454e+10,0.000000e+00,9.988454e+10,...,600.264517,479.521358,4953.130143,8424.353767,1.716544e+04,0.000000e+00,0.0000,2.190448e+10,NaN,2025-05-25 19:03:00+00:00
973,20182,Mobipad,MBP,mobipad,5,2022-05-18 12:38:21+00:00,"[launchpad, bnb-chain-ecosystem]",1.000000e+08,0.000000e+00,1.000000e+08,...,-1.886105,-3.814925,4.996650,36796.727749,3.679673e+04,0.000000e+00,0.0000,6.790549e+06,NaN,2025-05-25 19:03:00+00:00
974,33471,SORA GROK,GROK,sora-grok,1,2024-10-16 05:43:19+00:00,[ethereum-ecosystem],1.000000e+11,0.000000e+00,1.000000e+11,...,-98.583735,-98.121671,1244.358829,5563.341372,5.593531e+04,0.000000e+00,0.0000,1.633631e+09,NaN,2025-05-25 19:03:00+00:00
975,35487,Launch Coin on Believe,LAUNCHCOIN,launch-coin-on-believe,97,2025-01-27 06:31:18+00:00,"[solana-ecosystem, internet-capital-markets, b...",1.000000e+09,9.998743e+08,9.998743e+08,...,-2.861076,-19.991750,9834.423777,20959.967951,6.934382e+04,1.848381e+08,0.0000,1.848613e+08,NaN,2025-05-25 19:03:00+00:00


In [15]:
# listing only coins that are added between 2nd September 2011 and 23rd December 2015
date_added_df = listings_df[(listings_df['date_added'] > '02-09-2011') & (listings_df['date_added'] < '23-12-2015')]
date_added_df.reset_index(drop=True)



,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
0,1027,Ethereum,ETH,ethereum,10307,2015-08-07 00:00:00+00:00,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207260e+08,1.207260e+08,...,-1.414252,2.107912,39.794561,26.117982,-5.148470,3.038475e+11,9.0051,3.038475e+11,NaN,2025-05-25 19:03:00+00:00
1,825,Tether USDt,USDT,tether,129577,2015-02-25 00:00:00+00:00,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,1.527330e+11,1.560047e+11,...,0.016888,0.004997,-0.014991,0.028329,-0.013933,1.527815e+11,4.5280,1.560543e+11,NaN,2025-05-25 19:02:00+00:00
2,52,XRP,XRP,xrp,1614,2013-08-04 00:00:00+00:00,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.868601e+10,9.998611e+10,...,-1.849945,-4.035482,4.963800,-2.963648,-5.979773,1.351875e+11,4.0065,2.303573e+11,NaN,2025-05-25 19:03:00+00:00
3,74,Dogecoin,DOGE,dogecoin,1227,2013-12-15 00:00:00+00:00,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,1.494131e+11,1.494131e+11,...,-3.302346,-3.798250,20.798547,13.025218,-2.734797,3.278373e+10,0.9716,3.278373e+10,NaN,2025-05-25 19:03:00+00:00
4,512,Stellar,XLM,stellar,708,2014-08-05 00:00:00+00:00,"[medium-of-exchange, enterprise-solutions, dec...",5.000181e+10,3.111583e+10,5.000179e+10,...,-1.614864,-2.312966,-1.069160,-0.892082,-10.533556,8.793850e+09,0.2606,1.413134e+10,NaN,2025-05-25 19:03:00+00:00
5,328,Monero,XMR,monero,318,2014-05-21 00:00:00+00:00,"[mineable, pow, medium-of-exchange, privacy, r...",NaN,1.844674e+07,1.844674e+07,...,1.671921,20.162873,77.879766,84.963570,74.944460,7.546190e+09,0.2236,7.546190e+09,NaN,2025-05-25 19:03:00+00:00
6,2,Litecoin,LTC,litecoin,1371,2013-04-28 00:00:00+00:00,"[mineable, pow, scrypt, medium-of-exchange, la...",8.400000e+07,7.589320e+07,8.400000e+07,...,-2.242865,-4.494499,8.703144,2.798367,-23.611741,7.162124e+09,0.2123,7.927171e+09,NaN,2025-05-25 19:03:00+00:00
7,131,Dash,DASH,dash,478,2014-02-14 00:00:00+00:00,"[mineable, hybrid-pow-pos, x11, medium-of-exch...",1.890000e+07,1.227593e+07,1.227593e+07,...,-1.904565,-0.755880,2.808224,-3.736472,-9.212952,2.831903e+08,0.0084,4.359993e+08,NaN,2025-05-25 19:03:00+00:00
8,1042,Siacoin,SC,siacoin,122,2015-08-26 00:00:00+00:00,"[mineable, pow, blake2b, platform, distributed...",NaN,5.602564e+10,6.044816e+10,...,-2.813796,-1.003846,-7.072514,-1.411662,-14.433322,1.903159e+08,0.0056,2.053390e+08,NaN,2025-05-25 19:03:00+00:00
9,109,DigiByte,DGB,digibyte,167,2014-02-06 00:00:00+00:00,"[mineable, pow, multiple-algorithms, medium-of...",2.100000e+10,1.771002e+10,1.771002e+10,...,-2.555707,-6.000835,-4.192295,-11.186593,30.910422,1.752010e+08,0.0052,2.077480e+08,NaN,2025-05-25 19:03:00+00:00


In [16]:
# listing only the top 10 coins
top_10_df = listings_df[listings_df['cmc_rank'] <= 10]
top_10_df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
0,1,Bitcoin,BTC,bitcoin,12187,2010-07-13 00:00:00+00:00,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.986980e+07,1.986980e+07,...,-1.347124,2.810681,12.757457,24.610176,13.927691,2.135314e+12,63.2857,2.256771e+12,NaN,2025-05-25 19:03:00+00:00
1,1027,Ethereum,ETH,ethereum,10307,2015-08-07 00:00:00+00:00,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207260e+08,1.207260e+08,...,-1.414252,2.107912,39.794561,26.117982,-5.148470,3.038475e+11,9.0051,3.038475e+11,NaN,2025-05-25 19:03:00+00:00
2,825,Tether USDt,USDT,tether,129577,2015-02-25 00:00:00+00:00,"[stablecoin, asset-backed-stablecoin, usd-stab...",NaN,1.527330e+11,1.560047e+11,...,0.016888,0.004997,-0.014991,0.028329,-0.013933,1.527815e+11,4.5280,1.560543e+11,NaN,2025-05-25 19:02:00+00:00
3,52,XRP,XRP,xrp,1614,2013-08-04 00:00:00+00:00,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.868601e+10,9.998611e+10,...,-1.849945,-4.035482,4.963800,-2.963648,-5.979773,1.351875e+11,4.0065,2.303573e+11,NaN,2025-05-25 19:03:00+00:00
4,1839,BNB,BNB,bnb,2545,2017-07-25 00:00:00+00:00,"[marketplace, centralized-exchange, payments, ...",NaN,1.408881e+08,1.408881e+08,...,-1.484729,3.304452,10.219053,7.628989,4.982428,9.374372e+10,2.7783,9.374372e+10,NaN,2025-05-25 19:02:00+00:00
5,5426,Solana,SOL,solana,924,2020-04-10 00:00:00+00:00,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,5.202837e+08,6.015333e+08,...,-2.954381,0.757709,13.302747,24.985409,12.985334,8.934963e+10,2.6480,1.033028e+11,NaN,2025-05-25 19:03:00+00:00
6,3408,USDC,USDC,usd-coin,28186,2018-10-08 00:00:00+00:00,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,6.171563e+10,6.171563e+10,...,0.001908,-0.006457,-0.003849,-0.002124,-0.006248,6.170706e+10,1.8289,6.170706e+10,NaN,2025-05-25 19:03:00+00:00
7,74,Dogecoin,DOGE,dogecoin,1227,2013-12-15 00:00:00+00:00,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,1.494131e+11,1.494131e+11,...,-3.302346,-3.798250,20.798547,13.025218,-2.734797,3.278373e+10,0.9716,3.278373e+10,NaN,2025-05-25 19:03:00+00:00
8,2010,Cardano,ADA,cardano,1521,2017-10-01 00:00:00+00:00,"[dpos, pos, platform, research, smart-contract...",4.500000e+10,3.533451e+10,4.499450e+10,...,-1.176673,0.065855,3.763610,2.899820,2.716070,2.638633e+10,0.7820,3.360410e+10,NaN,2025-05-25 19:03:00+00:00
9,1958,TRON,TRX,tron,1178,2017-09-13 00:00:00+00:00,"[media, payments, tron-ecosystem, layer-1, dwf...",NaN,9.487783e+10,9.487783e+10,...,0.003916,2.114464,11.740144,17.936722,9.747367,2.572893e+10,0.7625,2.572893e+10,NaN,2025-05-25 19:03:00+00:00


In [17]:
bottom_10_df = listings_df.tail(10)
bottom_10_df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
4990,33623,Miners.Club,MBC,miners-club,2,2025-01-22 06:54:06+00:00,[bnb-chain-ecosystem],3.000000e+08,0.0,3.000000e+08,...,2.147191,27.393841,11.660586,-38.038784,-10.514182,0.0,0.0,3.631629e+06,NaN,2025-05-25 19:03:00+00:00
4991,34963,Wrapped MistCoin,WMC,wrapped-mistcoin,6,2024-12-31 06:34:02+00:00,"[ethereum-ecosystem, base-ecosystem]",NaN,0.0,5.446340e+05,...,-4.034579,-15.801615,7.236682,-7.172822,-12.723267,0.0,0.0,3.341647e+06,NaN,2025-05-25 19:03:00+00:00
4992,32527,Pepe of Solana,PEPE,pepe-of-solana,21,2024-08-02 04:45:36+00:00,"[memes, solana-ecosystem]",1.000000e+09,0.0,1.000000e+09,...,-6.615497,-7.089224,50.167961,59.308522,50.340337,0.0,0.0,1.327408e+04,NaN,2025-05-25 19:02:00+00:00
4993,33612,BOYS,CLUB,boys,2,2024-10-24 09:40:03+00:00,"[memes, solana-ecosystem]",1.000000e+11,0.0,1.000000e+11,...,-3.472403,-3.236548,-11.308486,-10.006481,-21.941143,0.0,0.0,6.540447e+06,NaN,2025-05-25 19:03:00+00:00
4994,27363,Sovryn Dollar,DLLR,sovryn-dollar,3,2023-09-06 13:28:41+00:00,[rsk-rbtc-ecosystem],0.000000e+00,0.0,0.000000e+00,...,-0.901263,1.421889,0.270166,0.245198,1.405890,0.0,0.0,0.000000e+00,NaN,2025-05-25 19:02:00+00:00
4995,25712,AiAkita,AIAKITA,aiakita,3,2023-05-22 21:50:37+00:00,[arbitrum-ecosystem],3.139079e+17,0.0,3.139079e+17,...,1.668990,-2.217800,-27.547658,-25.165560,-38.338088,0.0,0.0,3.265387e+04,NaN,2025-05-25 19:02:00+00:00
4996,36565,$SEND,$SEND,send,1,2025-05-20 09:58:34+00:00,"[memes, solana-ecosystem]",1.000000e+17,0.0,1.000000e+17,...,-36.750024,-96.188138,-96.188138,-96.188138,-96.188138,0.0,0.0,2.566771e+04,NaN,2025-05-25 19:03:00+00:00
4997,35067,BCGame Coin,BC,bcgame-coin,2,2025-01-07 06:53:23+00:00,[solana-ecosystem],NaN,0.0,1.000000e+10,...,-5.460912,102.958377,52.589969,74.535495,44.872525,0.0,0.0,6.516923e+07,NaN,2025-05-25 19:03:00+00:00
4998,24439,Silver,KAG,silver,20,2023-04-18 14:22:39+00:00,[],NaN,0.0,1.000000e+11,...,-0.673155,3.011141,-1.620380,-1.467504,2.186950,0.0,0.0,3.320438e+12,NaN,2025-05-25 19:03:00+00:00
4999,34891,DORA AI by Virtuals,DORA,dora-ai-by-virtuals,10,2024-12-27 03:41:42+00:00,"[ai-big-data, base-ecosystem, ai-agents, virtu...",1.000000e+09,0.0,1.000000e+09,...,-16.910973,-34.557707,31.708804,67.975332,39.396818,0.0,0.0,1.301463e+06,NaN,2025-05-25 19:03:00+00:00


In [18]:
top_10_price = top_10_df[['name','price']]
top_10_price = top_10_price.sort_values(by='price', ascending=False)
top_10_price

,name,price
0,Bitcoin,107465.299836
1,Ethereum,2516.836597
4,BNB,665.377027
5,Solana,171.732528
3,XRP,2.303573
2,Tether USDt,1.000318
6,USDC,0.999861
8,Cardano,0.746758
9,TRON,0.271180
7,Dogecoin,0.219417


In [19]:
fig = px.bar(top_10_price, 
             x='name',
             y='price',
             title='Top 10 Most Valuable Cryptocurrencies', 
             text='price',
             labels={'name': 'Cryptocurrency', 
                       'price': 'Price (USD)'},
             log_y=True
             )

fig.update_layout(
    height = 600,
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.show()

In [20]:
top_10_volume = listings_df.sort_values(by='volume_24h', ascending=False).head(10)
top_10_volume_df = top_10_volume[['name','volume_24h','symbol','market_cap','percent_change_24h','price']]
top_10_volume_df.reset_index(drop=True)


,name,volume_24h,symbol,market_cap,percent_change_24h,price
0,Tether USDt,6.350604e+10,USDT,1.527815e+11,0.016888,1.000318
1,Bitcoin,4.723863e+10,BTC,2.135314e+12,-1.347124,107465.299836
2,Dai,1.892487e+10,DAI,5.364525e+09,-0.004617,0.999840
3,Ethereum,1.392032e+10,ETH,3.038475e+11,-1.414252,2516.836597
4,USDC,8.409247e+09,USDC,6.170706e+10,0.001908,0.999861
5,Polyhedra Network,4.149006e+09,ZKJ,5.954663e+08,0.150888,2.033272
6,First Digital USD,3.729940e+09,FDUSD,1.643961e+09,-0.029289,0.998599
7,Solana,3.197687e+09,SOL,8.934963e+10,-2.954381,171.732528
8,XRP,1.928875e+09,XRP,1.351875e+11,-1.849945,2.303573
9,BNB,1.565767e+09,BNB,9.374372e+10,-1.484729,665.377027


In [21]:
fig = px.bar(top_10_volume_df,
             x='name',
             y='volume_24h',
             
             labels={'symbol': 'Symbol',
                     'volume_24h': '24h Trading Volume (USD)',
                     'price': 'Price',
                     'market_cap': 'Market_cap',
                     'percent_change_24h': 'Percent_change_24h',
                     'name': 'Cryptocurrency'},
             
             hover_data={'symbol':True,
                         'price': ':$,.2f',           
                         'market_cap': ':$,.0f',       
                         'percent_change_24h': ':.6f',
                         'volume_24h': ':,.2f'
                         },
             
             color='name',
             title = 'Top 10 coins by 24h trading volume',
             text='volume_24h',
             )

fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')

fig.update_layout(
    height = 600,
)

fig.show()

In [22]:
# listing the market dominance of top 10 coins
top_10_market_dominance = top_10_df[['name','market_cap_dominance','symbol']]
if top_10_market_dominance['market_cap_dominance'].sum() < 100:
    top_10_market_dominance = pd.concat([
        top_10_market_dominance,
        pd.DataFrame([{'name':'Remaining','market_cap_dominance': 100 - top_10_market_dominance['market_cap_dominance'].sum()}])
    ])
    
top_10_market_dominance


,name,market_cap_dominance,symbol
0,Bitcoin,63.2857,BTC
1,Ethereum,9.0051,ETH
2,Tether USDt,4.5280,USDT
3,XRP,4.0065,XRP
4,BNB,2.7783,BNB
5,Solana,2.6480,SOL
6,USDC,1.8289,USDC
7,Dogecoin,0.9716,DOGE
8,Cardano,0.7820,ADA
9,TRON,0.7625,TRX


In [23]:
# pie chart of market cap dominance of top 10 coins
fig = px.pie(top_10_market_dominance,
             values='market_cap_dominance',
             names='name',
             title='Crypto Market Cap Dominance',
             )
fig.update_traces(
    textposition='outside'
)

fig.show()

In [24]:
heatmap_df = listings_df[['name','percent_change_1h', 'percent_change_24h', 'percent_change_7d',
'percent_change_30d', 'percent_change_60d', 'percent_change_90d']].head(10)
heatmap_df.index = heatmap_df.index+1
heatmap_df


,name,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d
1,Bitcoin,0.113222,-1.347124,2.810681,12.757457,24.610176,13.927691
2,Ethereum,0.140873,-1.414252,2.107912,39.794561,26.117982,-5.148470
3,Tether USDt,-0.007662,0.016888,0.004997,-0.014991,0.028329,-0.013933
4,XRP,0.209174,-1.849945,-4.035482,4.963800,-2.963648,-5.979773
5,BNB,0.033105,-1.484729,3.304452,10.219053,7.628989,4.982428
6,Solana,0.420165,-2.954381,0.757709,13.302747,24.985409,12.985334
7,USDC,0.004922,0.001908,-0.006457,-0.003849,-0.002124,-0.006248
8,Dogecoin,0.136843,-3.302346,-3.798250,20.798547,13.025218,-2.734797
9,Cardano,0.399842,-1.176673,0.065855,3.763610,2.899820,2.716070
10,TRON,-0.048183,0.003916,2.114464,11.740144,17.936722,9.747367


In [25]:
# Reshaping the dataframe into long format 
time_windows = [
    'percent_change_1h', 'percent_change_24h', 'percent_change_7d',
    'percent_change_30d', 'percent_change_60d', 'percent_change_90d'
]
df_long = heatmap_df.melt(id_vars='name', 
                          value_vars=time_windows,
                          var_name='Time Window', 
                          value_name='Percent Change')

df_long



,name,Time Window,Percent Change
0,Bitcoin,percent_change_1h,0.113222
1,Ethereum,percent_change_1h,0.140873
2,Tether USDt,percent_change_1h,-0.007662
3,XRP,percent_change_1h,0.209174
4,BNB,percent_change_1h,0.033105
5,Solana,percent_change_1h,0.420165
6,USDC,percent_change_1h,0.004922
7,Dogecoin,percent_change_1h,0.136843
8,Cardano,percent_change_1h,0.399842
9,TRON,percent_change_1h,-0.048183


In [26]:
# pivot long format back to wide format
df_pivot = df_long.pivot(index='name', columns='Time Window', values='Percent Change')
col = df_pivot.pop('percent_change_7d') 
df_pivot.insert(2, 'percent_change_7d', col) 
df_pivot_rounded = df_pivot.round(2)
df_pivot_rounded


Time Window,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d
name,,,,,,
BNB,0.03,-1.48,3.30,10.22,7.63,4.98
Bitcoin,0.11,-1.35,2.81,12.76,24.61,13.93
Cardano,0.40,-1.18,0.07,3.76,2.90,2.72
Dogecoin,0.14,-3.30,-3.80,20.80,13.03,-2.73
Ethereum,0.14,-1.41,2.11,39.79,26.12,-5.15
Solana,0.42,-2.95,0.76,13.30,24.99,12.99
TRON,-0.05,0.00,2.11,11.74,17.94,9.75
Tether USDt,-0.01,0.02,0.00,-0.01,0.03,-0.01
USDC,0.00,0.00,-0.01,-0.00,-0.00,-0.01


In [27]:
# creating custom red/green palette
custom_scale = [
    [0.0, 'darkred'],
    [0.2, 'red'], 
    [0.4, 'lightcoral'],
    [0.5, 'white'],
    [0.6, 'lightgreen'],
    [0.8, 'green'],
    [1.0, 'darkgreen']
]

# creating heatmap 
fig = px.imshow(
    df_pivot_rounded,
    text_auto=True,
    color_continuous_scale=custom_scale,
    zmin=-max(abs(df_pivot_rounded.min().min()), abs(df_pivot_rounded.max().max())),  
    zmax=max(abs(df_pivot_rounded.min().min()), abs(df_pivot_rounded.max().max())),
    labels=dict(x="Time Window", y="Coin", color="Percent Change"),
    aspect="auto",
    origin='lower',
    title='Percent Change Heatmap'
)

fig.show()

In [28]:
coin_name = 'Ethereum'  
filtered_df = all_data_df[all_data_df['name'] == coin_name]
filtered_df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
1,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.470784,6.522769,49.489501,45.093927,13.864227,3.193864e+11,9.4025,3.193864e+11,NaN,2025-05-28T18:37:00.000Z
31,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.481385,6.746408,49.436191,44.976218,13.920724,3.195660e+11,9.4078,3.195660e+11,NaN,2025-05-28T18:38:00.000Z
61,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.484497,6.644787,49.442325,45.114236,14.006549,3.196741e+11,9.4110,3.196741e+11,NaN,2025-05-28T18:39:00.000Z
91,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.433930,6.605977,49.394016,45.062828,14.039155,3.197283e+11,9.4126,3.197283e+11,NaN,2025-05-28T18:40:00.000Z
121,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.195019,6.684185,49.371280,45.007293,13.937088,3.196764e+11,9.4092,3.196764e+11,NaN,2025-05-28T18:42:00.000Z
151,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.182753,6.363070,49.410177,45.031406,14.140215,3.196638e+11,9.4088,3.196638e+11,NaN,2025-05-28T18:43:00.000Z
181,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.090468,6.197112,49.724336,45.208487,14.238596,3.200684e+11,9.4208,3.200684e+11,NaN,2025-05-28T18:44:00.000Z
211,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.170117,6.254420,49.676456,45.186561,14.242245,3.200032e+11,9.4188,3.200032e+11,NaN,2025-05-28T18:45:00.000Z
241,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.170117,6.254420,49.676456,45.186561,14.242245,3.200032e+11,9.4188,3.200032e+11,NaN,2025-05-28T18:45:00.000Z
271,1027,Ethereum,ETH,ethereum,10316,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207252e+08,1.207252e+08,...,-1.116392,6.810365,49.537700,45.279733,14.092108,3.202050e+11,9.4160,3.202050e+11,NaN,2025-05-28T18:47:00.000Z


In [29]:
fig = px.line(filtered_df,
              x='last_updated.1', 
              y='price', 
              color='name', 
              title=f'{coin_name} Price Over Time', 
              labels={'last_updated.1': 'Time in minutes',
                      'symbol':'Symbol',
                      'price':'Price',
                      'name':'Cryptocurrency',
                      },
              
              hover_data={'symbol':True,
                          'price': ':,.2f',
              }
              
              )




fig.show()

In [30]:
grouped_df = all_data_df.groupby('name')[time_windows].mean()
grouped_df

,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d
name,,,,,,
Aave,-0.093771,-4.369434,6.849409,59.857015,56.515253,29.617872
Avalanche,-0.581615,-2.103495,1.393924,5.619307,15.389210,1.968323
BNB,0.285059,-0.268230,2.838011,14.089325,14.048416,13.116464
Bitcoin,0.164877,-2.164082,0.462113,14.400209,30.407347,27.173498
Bitcoin Cash,0.288880,-2.477317,1.905926,17.413818,34.911909,36.825462
Bitget Token,0.009704,-2.424074,0.943662,18.993070,13.507885,26.725276
Cardano,-0.027756,-2.798458,-1.226742,6.831257,11.016899,13.703699
Chainlink,-0.154324,-2.999367,-1.823966,4.978652,14.174179,1.146713
Dai,-0.005647,0.012225,0.009590,-0.012763,-0.003037,0.024885


In [31]:
# filtering out top 10 coins we want
coins=['Bitcoin','Ethereum','Avalanche','Aave','BNB','Stellar','Sui','Shiba Inu','Monero','Litecoin']
top_10_coins = all_data_df.loc[all_data_df['name'].isin(coins)].reset_index(drop=True)
top_10_mean = top_10_coins.groupby('name')[time_windows].mean().reset_index()
top_10_mean


,name,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d
0,Aave,-0.093771,-4.369434,6.849409,59.857015,56.515253,29.617872
1,Avalanche,-0.581615,-2.103495,1.393924,5.619307,15.389210,1.968323
2,BNB,0.285059,-0.268230,2.838011,14.089325,14.048416,13.116464
3,Bitcoin,0.164877,-2.164082,0.462113,14.400209,30.407347,27.173498
4,Ethereum,0.024563,-1.296509,6.573833,49.427517,45.092786,14.031668
5,Litecoin,-0.126900,-1.610542,-0.364877,11.828244,11.061670,-25.090171
6,Monero,0.633034,-11.288906,-9.711856,33.151002,60.440362,64.053704
7,Shiba Inu,0.017486,-2.979375,-2.960440,3.999246,11.707215,-3.000822
8,Stellar,0.017005,-2.221245,-2.044503,-0.529565,6.304811,-1.333628
9,Sui,-0.035798,-3.723849,-8.306998,0.148486,55.621894,25.632883


In [32]:
# converting df to long format for visualization
top_10_mean.columns = [col.replace('percent_change_','') for col in top_10_mean.columns]
time_windows = ['1h','24h','7d','30d','60d','90d']
long_top_10_mean = top_10_mean.melt(id_vars='name',value_vars=time_windows,value_name='Percent Change',var_name='Time Frame')
long_top_10_mean

,name,Time Frame,Percent Change
0,Aave,1h,-0.093771
1,Avalanche,1h,-0.581615
2,BNB,1h,0.285059
3,Bitcoin,1h,0.164877
4,Ethereum,1h,0.024563
5,Litecoin,1h,-0.126900
6,Monero,1h,0.633034
7,Shiba Inu,1h,0.017486
8,Stellar,1h,0.017005
9,Sui,1h,-0.035798


In [33]:
fig = px.line(long_top_10_mean, 
              x='Time Frame', 
              y='Percent Change', 
              color='name',
              markers=True,
              title='Percent Change over Time')

fig.update_layout(
    legend_title_text='Coin'
)

fig.show()

In [34]:
# grouping the top 15 coins by volume_24h
g_hist_df = hist_df.groupby('name')['volume_24h'].mean().reset_index(0)
g_hist_df


,name,volume_24h
0,Avalanche,5.974726e+08
1,BNB,1.974096e+09
2,Bitcoin,5.995046e+10
3,Cardano,9.693995e+08
4,Chainlink,6.504657e+08
5,Dogecoin,2.169179e+09
6,Ethereum,2.312198e+10
7,Hyperliquid,4.312177e+08
8,Solana,4.159297e+09
9,Stellar,2.444190e+08


In [35]:
g_hist_df = g_hist_df.sort_values('volume_24h', ascending=False)
fig = px.bar(g_hist_df,
             x='name',
             y='volume_24h', 
             title='Top cryptocurrency by 24h volume',
             labels={'volume_24h': '24h Trading Volume (USD)', 'name': 'Cryptocurrency'},
             color='volume_24h',
             color_continuous_scale='Plasma',
             text = 'volume_24h',
             height=600)




fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')

fig.update_layout(yaxis_type="log")

fig.show()


In [36]:
volume_price_df = hist_df.groupby('name')[['volume_24h','price','market_cap']].mean().reset_index()
volume_price_df

,name,volume_24h,price,market_cap
0,Avalanche,5.974726e+08,21.585623,9.091775e+09
1,BNB,1.974096e+09,669.754721,9.436023e+10
2,Bitcoin,5.995046e+10,105546.241863,2.097425e+12
3,Cardano,9.693995e+08,0.702710,2.483217e+10
4,Chainlink,6.504657e+08,14.348752,9.428564e+09
5,Dogecoin,2.169179e+09,0.202803,3.031469e+10
6,Ethereum,2.312198e+10,2595.927525,3.133919e+11
7,Hyperliquid,4.312177e+08,32.337075,1.079826e+10
8,Solana,4.159297e+09,161.467771,8.411222e+10
9,Stellar,2.444190e+08,0.274174,8.539208e+09


In [37]:
fig = px.scatter(volume_price_df, 
                x='volume_24h', 
                y='price',
                color='name',
                title='Cryptocurrency: Volume vs Price Analysis',
                hover_data={'market_cap': ':$,.0f', 'volume_24h': ':$,.0f', 'price': ':$,.2f'},
                labels={'volume_24h': '24h Trading Volume (USD)', 
                       'price': 'Price (USD)'},
                size_max=50,)

fig.update_layout(
    xaxis_tickformat='$,.1s',
    yaxis_tickformat='$,.0f',
    height=600,
    showlegend=True,
    legend=dict(orientation="v", x=1.02, y=1,),
)

fig.update_layout(xaxis_type="log", yaxis_type="log")

fig.show()


In [38]:
fig = px.scatter(listings_df, 
                x='volume_24h', 
                y='price',
                trendline='ols',
                title='Cryptocurrency: Volume vs Price Analysis',
                hover_data={'market_cap': ':$,.0f', 'volume_24h': ':$,.0f', 'price': ':$,.2f'},
                labels={'volume_24h': '24h Trading Volume (USD)', 
                       'price': 'Price (USD)'},)

fig.update_layout(
    height=600,
)


fig.show()

In [40]:
# calculating volume to market cap ratio
volume_price_df['volume_to_market_cap'] = volume_price_df['volume_24h'] / volume_price_df['market_cap']
top_ratio = volume_price_df.sort_values(by='volume_to_market_cap', ascending=False)
fig = px.bar(
    top_ratio,
    x='name',
    y='volume_to_market_cap',
    title='Top 10 Coins by Volume/Market Cap Ratio',
    text='volume_to_market_cap'
)

fig.update_layout(
    height=600,
)


fig.update_traces(texttemplate='%{text:.6f}', textposition='outside')
fig.show()

In [41]:
# adding new moving average columns to the dataframe and calculating 15/30mins moving average
coin='Bitcoin'
rolling_avg_15m_df = hist_df[hist_df['name'] == coin].copy()
rolling_avg_30m_df = rolling_avg_15m_df.copy()
rolling_avg_15m_df['rolling_average'] = rolling_avg_15m_df['price'].rolling(window=3).mean()
rolling_avg_30m_df['rolling_average'] = rolling_avg_30m_df['price'].rolling(window=6).mean()
rolling_avg_15m_df



,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1,rolling_average
0,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-3.081134,11.704631,27.893873,24.887847,2.095713e+12,63.0732,2.214664e+12,NaN,2025-05-30T13:24:00.000Z,NaN
15,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-2.796150,11.755397,28.033323,25.165252,2.099928e+12,63.2653,2.219119e+12,NaN,2025-05-30T13:29:00.000Z,NaN
30,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-3.569435,11.667075,28.872501,24.848617,2.097064e+12,63.1069,2.216092e+12,NaN,2025-05-30T13:34:00.000Z,105553.569825
45,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-3.461149,12.099746,28.454137,24.950135,2.098757e+12,63.1991,2.217881e+12,NaN,2025-05-30T13:38:00.000Z,105604.635962
60,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-2.983588,12.664501,29.115308,25.304718,2.102403e+12,63.2516,2.221734e+12,NaN,2025-05-30T13:44:00.000Z,105646.156843
75,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-2.981044,12.930895,29.396744,25.598273,2.105580e+12,63.2926,2.225091e+12,NaN,2025-05-30T13:49:00.000Z,105788.995386
90,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-3.448916,13.173603,27.636368,25.147572,2.098387e+12,63.0444,2.217490e+12,NaN,2025-05-30T13:55:00.000Z,105782.780580
105,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-3.749960,13.384948,27.937279,24.926891,2.097200e+12,63.1973,2.216235e+12,NaN,2025-05-30T13:59:00.000Z,105695.493553
120,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-3.990439,13.036881,27.557979,24.853262,2.098227e+12,63.3080,2.217321e+12,NaN,2025-05-30T14:04:00.000Z,105572.151247
135,1,Bitcoin,BTC,bitcoin,12204,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,19872075.0,19872075.0,...,-3.561283,13.328091,27.314563,25.029440,2.102896e+12,63.4078,2.222255e+12,NaN,2025-05-30T14:09:00.000Z,105647.798893


In [42]:
fig = px.line(rolling_avg_15m_df, 
              x='last_updated.1', 
              y=['price','rolling_average'], 
              title=f'{coin} 15 Minutes Rolling Average', 
              labels={'last_updated.1': 'Time'}, 
                    )
fig.show()

In [43]:
fig = px.line(rolling_avg_30m_df, 
              x='last_updated.1', 
              y=['price','rolling_average'], 
              title=f'{coin} 30 Minutes Rolling Average', 
              labels={'last_updated.1': 'Time'}, 
                    )
fig.show()

In [44]:
# creating correlation heatmap dataframe
change_cols = ['name', 'percent_change_1h', 'percent_change_24h', 
               'percent_change_7d', 'percent_change_30d', 
               'percent_change_60d', 'percent_change_90d']

change_cols2 = ['percent_change_1h', 'percent_change_24h', 
               'percent_change_7d', 'percent_change_30d', 
               'percent_change_60d', 'percent_change_90d']

coins=['Bitcoin','Ethereum','Avalanche','Aave','BNB','Stellar','Sui','Shiba Inu','Monero','Litecoin','Dogecoin','Solana','Chainlink','BCGame Coin', 'AiAkita']

corr_df = listings_df.loc[listings_df['name'].isin(coins)].reset_index(drop=True)
corr_df = corr_df[change_cols].dropna()
corr_df = corr_df.groupby('name')[change_cols2].mean().reset_index()




In [45]:
# Transpose to get coins as columns
corr_map = corr_df.set_index('name').T.corr()
corr_map

name,Aave,AiAkita,Avalanche,BCGame Coin,BNB,Bitcoin,Chainlink,Dogecoin,Ethereum,Litecoin,Monero,Shiba Inu,Solana,Stellar,Sui
name,,,,,,,,,,,,,,,
Aave,1.000000,-0.655052,0.483541,0.533276,0.959415,0.801129,0.350583,0.925105,0.936174,0.465217,0.846275,0.316929,0.816973,0.127594,0.428194
AiAkita,-0.655052,1.000000,0.188370,-0.345789,-0.784894,-0.820969,0.386394,-0.475038,-0.403192,0.326768,-0.944279,-0.236264,-0.810801,0.632763,-0.581533
Avalanche,0.483541,0.188370,1.000000,0.317993,0.319928,0.327906,0.855236,0.559208,0.597593,0.770376,0.110295,0.026105,0.367826,0.781367,0.295312
BCGame Coin,0.533276,-0.345789,0.317993,1.000000,0.607401,0.496410,-0.089225,0.222093,0.329362,0.006477,0.497155,-0.412101,0.451185,-0.120967,0.227320
BNB,0.959415,-0.784894,0.319928,0.607401,1.000000,0.809702,0.143409,0.829113,0.826925,0.243767,0.909454,0.333681,0.818043,-0.104461,0.401289
Bitcoin,0.801129,-0.820969,0.327906,0.496410,0.809702,1.000000,-0.001536,0.630390,0.592072,0.017575,0.936727,0.000458,0.995937,-0.224939,0.859699
Chainlink,0.350583,0.386394,0.855236,-0.089225,0.143409,-0.001536,1.000000,0.594616,0.615119,0.947341,-0.133650,0.366318,0.062848,0.945695,-0.060447
Dogecoin,0.925105,-0.475038,0.559208,0.222093,0.829113,0.630390,0.594616,1.000000,0.983168,0.671468,0.668343,0.567997,0.675678,0.361477,0.296556
Ethereum,0.936174,-0.403192,0.597593,0.329362,0.826925,0.592072,0.615119,0.983168,1.000000,0.727470,0.628530,0.465190,0.628196,0.420439,0.231389


In [46]:
# correlation heatmap
fig = px.imshow(
    corr_map,
    text_auto=".2f",
    color_continuous_scale='RdBu',
    zmin=-1, zmax=1,
    title='Cryptocurrency Percent Change Correlation Heatmap',
    labels=dict(color='Correlation')
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='',
    coloraxis_colorbar=dict(title="Corr"),
    height = 700,

)
fig.show()

In [47]:
# identify top 10 sectors
all_tags = list(chain.from_iterable(listings_df['tags']))
tag_counts = Counter(all_tags)
top_10_tags_list = tag_counts.most_common(10)
top_10_tags = [tag for tag, _ in top_10_tags_list]
top_10_tags

['ethereum-ecosystem',
 'bnb-chain-ecosystem',
 'memes',
 'solana-ecosystem',
 'defi',
 'gaming',
 'polygon-ecosystem',
 'ai-big-data',
 'base-ecosystem',
 'collectibles-nfts']

In [48]:
top_10_tags_df = listings_df[listings_df['tags'].apply(lambda tags: any(tag in top_10_tags for tag in tags))]
top_10_tags_df


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
1,1027,Ethereum,ETH,ethereum,10307,2015-08-07 00:00:00+00:00,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.207260e+08,1.207260e+08,...,-1.414252,2.107912,39.794561,26.117982,-5.148470,3.038475e+11,9.0051,3.038475e+11,NaN,2025-05-25 19:03:00+00:00
4,1839,BNB,BNB,bnb,2545,2017-07-25 00:00:00+00:00,"[marketplace, centralized-exchange, payments, ...",NaN,1.408881e+08,1.408881e+08,...,-1.484729,3.304452,10.219053,7.628989,4.982428,9.374372e+10,2.7783,9.374372e+10,NaN,2025-05-25 19:02:00+00:00
5,5426,Solana,SOL,solana,924,2020-04-10 00:00:00+00:00,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,5.202837e+08,6.015333e+08,...,-2.954381,0.757709,13.302747,24.985409,12.985334,8.934963e+10,2.6480,1.033028e+11,NaN,2025-05-25 19:03:00+00:00
7,74,Dogecoin,DOGE,dogecoin,1227,2013-12-15 00:00:00+00:00,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,1.494131e+11,1.494131e+11,...,-3.302346,-3.798250,20.798547,13.025218,-2.734797,3.278373e+10,0.9716,3.278373e+10,NaN,2025-05-25 19:03:00+00:00
10,32196,Hyperliquid,HYPE,hyperliquid,82,2024-07-15 10:14:50+00:00,"[decentralized-exchange-dex-token, defi, deriv...",1.000000e+09,3.339282e+08,9.999939e+08,...,10.402405,46.330999,109.157446,181.415360,80.532379,1.287975e+10,0.3817,3.857042e+10,NaN,2025-05-25 19:03:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,32527,Pepe of Solana,PEPE,pepe-of-solana,21,2024-08-02 04:45:36+00:00,"[memes, solana-ecosystem]",1.000000e+09,0.000000e+00,1.000000e+09,...,-6.615497,-7.089224,50.167961,59.308522,50.340337,0.000000e+00,0.0000,1.327408e+04,NaN,2025-05-25 19:02:00+00:00
4993,33612,BOYS,CLUB,boys,2,2024-10-24 09:40:03+00:00,"[memes, solana-ecosystem]",1.000000e+11,0.000000e+00,1.000000e+11,...,-3.472403,-3.236548,-11.308486,-10.006481,-21.941143,0.000000e+00,0.0000,6.540447e+06,NaN,2025-05-25 19:03:00+00:00
4996,36565,$SEND,$SEND,send,1,2025-05-20 09:58:34+00:00,"[memes, solana-ecosystem]",1.000000e+17,0.000000e+00,1.000000e+17,...,-36.750024,-96.188138,-96.188138,-96.188138,-96.188138,0.000000e+00,0.0000,2.566771e+04,NaN,2025-05-25 19:03:00+00:00
4997,35067,BCGame Coin,BC,bcgame-coin,2,2025-01-07 06:53:23+00:00,[solana-ecosystem],NaN,0.000000e+00,1.000000e+10,...,-5.460912,102.958377,52.589969,74.535495,44.872525,0.000000e+00,0.0000,6.516923e+07,NaN,2025-05-25 19:03:00+00:00


In [49]:
tag_growth = defaultdict(list)

for _, row in top_10_tags_df.iterrows():
    for tag in row['tags']:
        if tag in top_10_tags:
            tag_growth[tag].append(row['percent_change_90d'])

# Calculate average growth for each top tag
avg_growth_per_tag = {tag: sum(values) / len(values) for tag, values in tag_growth.items()}
sorted_avg_growth = sorted(avg_growth_per_tag.items(), key=lambda x: x[1], reverse=True)
sorted_avg_growth


[('base-ecosystem', 16121.157376753685),
 ('memes', 14116.951362300362),
 ('bnb-chain-ecosystem', 4555.794253056733),
 ('ethereum-ecosystem', 2427.775025557274),
 ('solana-ecosystem', 99.10167192302661),
 ('defi', -1.425412602233676),
 ('collectibles-nfts', -10.370159821299735),
 ('gaming', -13.022613227100372),
 ('ai-big-data', -14.738389337089714),
 ('polygon-ecosystem', -15.797845455397725)]

In [50]:
avg_growth_df = pd.DataFrame(sorted_avg_growth, columns=["Tag", "Average_90d_Growth(%)"])
fig = px.bar(avg_growth_df,
             x="Tag",
             y="Average_90d_Growth(%)",
             title="Top 10 Tags by Average 90-Day Growth(%)",
             text_auto='.2f',
             )

fig.update_layout(height=600)
fig.show()

In [51]:
# Coins added in the last 180 days
recent_coins = listings_df[listings_df['date_added'] > datetime.now(timezone.utc) - timedelta(days=180)]

# Filter ones with positive performance
recent_gainers = recent_coins[recent_coins['percent_change_90d'] > 0].sort_values(by='percent_change_90d', ascending=False)
recent_gainers

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated.1
3197,36562,Trump Dinner,DINNER,trump-dinner,1,2025-05-20 09:54:02+00:00,"[memes, base-ecosystem]",2.200000e+11,0.000000e+00,2.200000e+11,...,1700.249891,6.597989e+06,6.597989e+06,6.597989e+06,6.597989e+06,0.000000e+00,0.0,3.723398e+10,NaN,2025-05-25 19:03:00+00:00
1375,35964,Mint Token,MT,mint-token,2,2025-03-07 07:45:06+00:00,"[platform, communications-social-media, dapp, ...",1.000000e+09,6.389244e+08,6.389244e+08,...,-1.568817,-8.796309e-01,3.250193e+00,2.272887e+01,2.524577e+05,4.520847e+06,0.0,7.075716e+06,NaN,2025-05-25 19:03:00+00:00
244,35487,Launch Coin on Believe,LAUNCHCOIN,launch-coin-on-believe,97,2025-01-27 06:31:18+00:00,"[solana-ecosystem, internet-capital-markets, b...",1.000000e+09,9.998743e+08,9.998743e+08,...,-2.861076,-1.999175e+01,9.834424e+03,2.095997e+04,6.934382e+04,1.848381e+08,0.0,1.848613e+08,NaN,2025-05-25 19:03:00+00:00
4175,36525,BOME GROK,GROK,bome-grok,1,2025-05-16 06:57:14+00:00,"[memes, bnb-chain-ecosystem]",1.000000e+09,0.000000e+00,1.000000e+09,...,-93.265713,4.993099e+00,9.610495e+03,9.610495e+03,9.610495e+03,0.000000e+00,0.0,9.861281e+06,NaN,2025-05-25 19:03:00+00:00
4625,35556,HODL,HODL,we-hodl,4,2025-01-31 08:28:31+00:00,[solana-ecosystem],NaN,0.000000e+00,8.500000e+08,...,-40.695173,5.330809e+02,3.485474e+03,3.694027e+03,2.841141e+03,0.000000e+00,0.0,4.092736e+05,NaN,2025-05-25 19:02:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,36039,Frax USD,FRXUSD,frax-usd,6,2025-03-16 04:43:36+00:00,"[stablecoin, usd-stablecoin, fiat-stablecoin]",NaN,7.667809e+07,7.667809e+07,...,-0.025694,-3.609551e-02,8.089010e-03,2.815667e-02,1.213015e-01,7.664408e+07,0.0,7.664408e+07,NaN,2025-05-25 19:03:00+00:00
788,35976,Aegis YUSD,YUSD,aegis-yusd,14,2025-03-08 12:18:51+00:00,"[stablecoin, dao, usd-stablecoin]",NaN,2.442938e+07,2.442938e+07,...,-0.017132,3.108880e-02,1.057752e-02,8.611900e-02,1.101059e-01,2.444347e+07,0.0,2.444347e+07,NaN,2025-05-25 19:03:00+00:00
3362,36552,LUSD,LUSD,lusd,3,2025-05-20 06:29:03+00:00,[bnb-chain-ecosystem],1.000000e+12,0.000000e+00,1.000000e+12,...,0.013531,5.205778e-02,5.205778e-02,5.205778e-02,5.205778e-02,0.000000e+00,0.0,1.000260e+12,NaN,2025-05-25 19:03:00+00:00
4002,36572,EuroUnion,EURC,eurounion,1,2025-05-21 08:13:23+00:00,"[ethereum-ecosystem, arbitrum-ecosystem, bnb-c...",2.000000e+10,0.000000e+00,2.000000e+10,...,0.015376,2.172413e-02,2.172413e-02,2.172413e-02,2.172413e-02,0.000000e+00,0.0,2.301318e+10,NaN,2025-05-25 19:03:00+00:00
